In [ ]:
import sys

sys.path.append("../src")

## Hydra-zen

### Generate Hydra compatible cfg

In [ ]:
import omegaconf

# root = pyrootutils.setup_root("/root/workspace/EP2", pythonpath=True)
cfg = omegaconf.OmegaConf.load("../configs/callbacks/eqprop_track.yaml")

In [ ]:
omegaconf.OmegaConf.register_new_resolver(
    "double", lambda condition, false_value: 2 * false_value if condition else false_value
)

In [ ]:
cfg.net.input_size

In [ ]:
print(omegaconf.OmegaConf.to_yaml(cfg))

In [ ]:
from hydra.utils import instantiate

instantiate(cfg)

In [ ]:
import hydra_zen

hydra_zen.instantiate(cfg)

### Make Hierachial configs

see https://github.com/mit-ll-responsible-ai/hydra-zen-examples

In [ ]:
import math

import pytorch_lightning as pl
import torch as tr
from hydra_zen import builds, instantiate, make_config, make_custom_builds_fn
from torch.optim import Adam
from torch.utils.data import DataLoader
from zen_model import UniversalFuncModule, single_layer_nn

pbuilds = make_custom_builds_fn(zen_partial=True, populate_full_signature=True)

OptimConf = pbuilds(Adam)

LoaderConf = pbuilds(DataLoader, batch_size=25, shuffle=True, drop_last=True)

ModelConf = builds(single_layer_nn, num_neurons=10)

# configure our lightning module
LitConf = pbuilds(
    UniversalFuncModule,
    model=ModelConf,
    target_fn=tr.cos,
    training_domain=builds(tr.linspace, start=-2 * math.pi, end=2 * math.pi, steps=1000),
)

TrainerConf = builds(pl.Trainer, max_epochs=100)

ExperimentConfig = make_config(
    optim=OptimConf,
    dataloader=LoaderConf,
    lit_module=LitConf,
    trainer=TrainerConf,
    seed=1,
)


def task_function(cfg):
    # cfg: ExperimentConfig
    pl.seed_everything(cfg.seed)

    obj = instantiate(cfg)

    # finish instantiating the lightning module, data-loader, and optimizer
    lit_module = obj.lit_module(dataloader=obj.dataloader, optim=obj.optim)

    # train the model
    obj.trainer.fit(lit_module)

    # evaluate the model over the domain to assess the fit
    data = lit_module.training_domain
    final_eval = lit_module.forward(data.reshape(-1, 1))
    final_eval = final_eval.detach().cpu().numpy().ravel()

    # return the final evaluation of our model:
    # a shape-(N,) numpy-array
    return final_eval

# Hydra

In [ ]:
import pyrootutils
from hydra import compose, initialize
from omegaconf import open_dict


def get_cfg():
    with initialize(version_base="1.2", config_path="../configs"):
        cfg = compose(config_name="train.yaml", return_hydra_config=True, overrides=[])
        with open_dict(cfg):
            cfg.paths.root_dir = str(pyrootutils.find_root())
            cfg.trainer.max_epochs = 1
            cfg.trainer.limit_train_batches = 0.01
            cfg.trainer.limit_val_batches = 0.1
            cfg.trainer.limit_test_batches = 0.1
            cfg.trainer.accelerator = "cpu"
            cfg.trainer.devices = 1
            cfg.datamodule.num_workers = 0
            cfg.datamodule.pin_memory = False
            cfg.datamodule.batch_size = 1
            cfg.extras.print_config = False
            cfg.extras.enforce_tags = False
            cfg.logger = None

        return cfg

In [ ]:
cfg.data

In [ ]:
import hydra

model = hydra.utils.instantiate(cfg.model)